In [ ]:
!pip install transformers
!pip install accelerate
!pip install bitsandbytes
!pip install sentencepiece
!pip install torch
!pip install datasets
!pip install evaluate
!pip install hugginface_hub
!pip install lm-zoo

In [2]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) n
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [3]:
!pip install datasets
!pip install evaluate
!pip install numpy
from transformers import LlamaForCausalLM, LlamaTokenizer, AutoTokenizer
import torch

import datasets
import evaluate
import numpy as np

from google.colab import userdata
my_secret_key = userdata.get('HF_TOKEN')

tokenizer_7b = AutoTokenizer.from_pretrained("meta-llama/Llama-2-7b-hf")

tokenizer_7b_chat = AutoTokenizer.from_pretrained("meta-llama/Llama-2-7b-chat-hf")

model_7b = LlamaForCausalLM.from_pretrained(
    "meta-llama/Llama-2-7b-hf",
    load_in_4bit=True,
    device_map="auto",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16
)

model_7b_chat = LlamaForCausalLM.from_pretrained(
    "meta-llama/Llama-2-7b-chat-hf",
    load_in_4bit=True,
    device_map="auto",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16
)

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

tokenizer_config.json:   0%|          | 0.00/776 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

...

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

In [4]:
import gzip
import shutil

with gzip.open('Sts.gz', 'rb') as f_in:
    with open('sts.txt', 'wb') as f_out:
        shutil.copyfileobj(f_in, f_out)

def read_sts_dataset(filename):
    with open(filename, 'r') as f:
        sts_data = f.readlines()
    return sts_data

sts_data = read_sts_dataset('sts.txt')


In [5]:
def parse_file(file_path):
    data = []
    with open(file_path, 'r', encoding='utf-8') as file:
        for line in file:
            parts = line.strip().split('\t')
            if len(parts) >= 6:
                entry = {
                    'score': float(parts[4]),
                    'sentence1': parts[5],
                    'sentence2': parts[6]
                }
                data.append(entry)

    return data

[{'score': 2.5, 'sentence1': 'A girl is styling her hair.', 'sentence2': 'A girl is brushing her hair.'}, {'score': 3.6, 'sentence1': 'A group of men play soccer on the beach.', 'sentence2': 'A group of boys are playing soccer on the beach.'}, {'score': 5.0, 'sentence1': "One woman is measuring another woman's ankle.", 'sentence2': "A woman measures another woman's ankle."}, {'score': 4.2, 'sentence1': 'A man is cutting up a cucumber.', 'sentence2': 'A man is slicing a cucumber.'}, {'score': 1.5, 'sentence1': 'A man is playing a harp.', 'sentence2': 'A man is playing a keyboard.'}, {'score': 1.8, 'sentence1': 'A woman is cutting onions.', 'sentence2': 'A woman is cutting tofu.'}, {'score': 3.5, 'sentence1': 'A man is riding an electric bicycle.', 'sentence2': 'A man is riding a bicycle.'}, {'score': 2.2, 'sentence1': 'A man is playing the drums.', 'sentence2': 'A man is playing the guitar.'}, {'score': 2.2, 'sentence1': 'A man is playing guitar.', 'sentence2': 'A lady is playing the gu

In [7]:
dataset = parse_file("sts.txt")
first_150_pairs = dataset[:150]

def generate_similarity_score(model_name, sentence1, sentence2):
    prompt = f"Rate the similarity of these two sentences on a scale from 0 to 5:\nSentence 1: {sentence1}\nSentence 2: {sentence2}. Only give the number as a response."
    model_response = invoke_model(model_name, prompt)
    predicted_score = parse_model_response(model_response)

    return predicted_score

def invoke_model(model, prompt, tokenizer):
    inputs = tokenizer.encode(prompt, return_tensors="pt")
    outputs = model.generate(inputs, max_length=50, num_return_sequences=1)
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return response

def parse_model_response(response):
    try:
        score = float(response)
    except ValueError:
        score = -1
    return score
